<a href="https://colab.research.google.com/github/jnramirezg/medio_continuo/blob/main/codigo/32-analisis_viga_en_voladizo_airy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis viga en voladizo con Airy (sección 5.14)

|Quién | Fecha | Qué hizo |
| ---  | ---   | ---      |
|Juan Nicolás Ramírez, <jnramirezg@unal.edu.co> | 2022-03-25 | Conversión de código de MAXIMA a Python|

[Código original](https://github.com/diegoandresalvarez/solidos/blob/master/archivos/codigo/cap_05/5.14_ejer_10_viga_voladizo_Airy.mac)

Importamos las librerias y funciones necesarias:

In [1]:
from sympy import symbols, diff, integrate, solve, factor, expand, collect, Rational, Add
import sympy as sp

# Para imprimir bonito
sp.init_printing()
from IPython.display import Math                 
def imprimir (texto, variable):
    return Math(texto +  rf'{sp.latex(variable)}')

Creamos las variables simbólicas:

In [2]:
x, y, z = symbols("x,y,z")               # Coordenadas.
C1, C2, C3, C4 = symbols("C1,C2,C3,C4")  # Constantes de la función de tensión de Airy.
K2 = symbols("K_2")                      # Conste de TP o DP de la ecuación biarmónica.
c, L, q = symbols("c,L,q")               # Propiedades de la viga.
E, nu = symbols("E, nu")                 # Propiedades del material.
u0, v0, w0 = symbols("u0, v0, omega_0")  # Constante de integración.

In [3]:
t = 1  # [m]: espesor.
V = 0  # V=0, ya que no se considerarán las fuerzas másicas.

Proponemos la función de tensión de Airy:

In [4]:
phi = C1*(L-x)**2 + C2*(L-x)**2*y + C3*y**3 + C4*y**5 - 5*C4*(L-x)**2*y**3
imprimir(r"\phi(x,y) = ", phi)

<IPython.core.display.Math object>

Definimos el laplaciano y el biarmónico de una función $f$:

In [5]:
lapl = lambda f: diff(f,x,2) + diff(f,y,2)
biar = lambda f: lapl(lapl(f))

Verificamos que se satisfaga la ecuación $\nabla^4\phi=K_2\nabla^2V$

In [6]:
if biar(phi) - K2*lapl(V) == 0:
    print("La ecuación biarmónica se satisface.")
else:
    print("La ecuación NO biarmónica se satisface.")

La ecuación biarmónica se satisface.


Definimos los esfuerzos utilizando la función de tensión de Airy:
$$\sigma_x(x,y)=\frac{\partial^2 \phi(x,y)}{\partial y^2} + V(x,y)$$
$$\sigma_x(x,y)=\frac{\partial^2 \phi(x,y)}{\partial x^2} + V(x,y)$$
$$\tau_{xy}(x,y)=-\frac{\partial^2 \phi(x,y)}{\partial x \partial y}$$

In [7]:
sx  =  diff(phi, y,2) + V
sy  =  diff(phi, x,2) + V
txy = -diff(phi, x,1, y,1)

Definimos la fuerza cortante, el momento flector y la fuerza axial:
$$V(x) = -\int_{-c}^{c}\int_{0}^{t}{\tau_{xy}(x,y,z)}dzdy$$
$$M(x) = -\int_{-c}^{c}\int_{0}^{t}{y\sigma_x(x,y,z)}dzdy$$
$$f_{axial}(x) = -\int_{-c}^{c}\int_{0}^{t}{\sigma_x(x,y,z)}dzdy=0$$

In [8]:
V   = lambda xx: -integrate(txy.subs(x,xx),   (z,0,t), (y,-c,c))
M   = lambda xx: -integrate( sx.subs(x,xx)*y, (z,0,t), (y,-c,c))
fax =             integrate( sx,              (z,0,t), (y,-c,c))

Establecemos las condiciones de frontera:

In [9]:
e1 = txy.subs(y, +c) - 0    # =0
e2 = txy.subs(y, -c) - 0    # =0
e3 =  sy.subs(y, +c) - q/t  # =0
e4 =  sy.subs(y, -c) - 0    # =0

e5 = V( 0) - (-q*L)      # =0
e6 = V(+L) - 0           # =0
e7 = M( 0) - (q*L**2/2)  # =0
e8 = M(+L) - 0           # =0
e9 = fax   - 0           # =0 para todo x en [−L, L]

Utilizando las ecuaciones ```e1``` a ```e9``` encontramos las constantes ```C1```, ```C2```, ```C3``` y ```C4```:

In [10]:
const = solve([e1,e2,e3,e4,e5,e6,e7,e8,e9], [C1,C2,C3,C4])

Reemplazamos ```C1```, ```C2```, ```C3``` y ```C4``` en los esfuerzos ```sx```, ```sy``` y ```txy```:

In [11]:
sx  = sx.subs(const)
sy  = sy.subs(const)
txy = txy.subs(const)
imprimir(r"\sigma_x=", factor(sx,q))

<IPython.core.display.Math object>

Definimos el módulo de cortante:

In [12]:
G = E/(2*(1+nu))

Usamos las ecuaciones de ley de Hooke de tensión plana para isótropos para calcular las deformaciones:

In [13]:
ex  = (1/E)*(sx - nu*sy)
ey  = (1/E)*(sy - nu*sx)
gxy = (1/G)*txy

De las ecuaciones:

$$u(x,y)=\int\varepsilon_x(x',y)dx'+f(y)$$
$$v(x,y)=\int\varepsilon_y(x,y')dy'+g(x)$$
$$\frac{\partial f(y)}{\partial y} + \frac{\partial g(x)}{\partial x}=\gamma_{xy}(x,y)-\frac{\partial}{\partial y}\left( \int\varepsilon_x(x',y)dx'\right) -\frac{\partial}{\partial x}\left(\int\varepsilon_y(x,y')dy' \right)$$

Usamos la tercera ecuación:

In [14]:
df_dy_dg_dx = gxy - diff(integrate(ex,x),y) - diff(integrate(ey,y),x)

Descomponemos la ecuación diferencial en:
$$\frac{\partial f(y)}{\partial y} + \omega_0=0$$
$$\frac{\partial g(x)}{\partial x} - \omega_0=0$$

In [15]:
df_dy = df_dy_dg_dx.subs(x,0) + w0
dg_dx = df_dy_dg_dx.subs(y,0) - w0

De esta forma:

$$\displaystyle \frac{\partial f(y)}{\partial y}+\frac{\partial g(x)}{\partial x}= \underbrace{\omega_0}_{\frac{\partial f(y)}{\partial y}} + \underbrace{\frac{q x \left(- 15 L^{2} - 15 c^{2} \nu - 24 c^{2} + 5 x^{2}\right)}{20 E c^{3}} - \omega_0}_{\frac{\partial g(x)}{\partial x}}$$


Ahora, calculamos ```f``` y ```g``` (sin considerar aún sus constantes de integración):

In [16]:
f = integrate(df_dy, y)
g = integrate(dg_dx, x)

Finalmente, calculamos los desplamientos ```u``` y ```v``` (con las constantes de integración u0 y v0, que consideran el aporte de ```f``` y ```g```):

In [17]:
u = integrate(ex, x) + f + u0
v = integrate(ey, y) + g + v0

Sólo resta encontrar los valores de ```u0```, ```v0``` y ```w0``` a partir de las condiciones de frontera y el conocimiento  de que en el empotramiento (L=0) de la viga los desplazamientos y la rotación som nulos:

In [18]:
rot = (diff(v,x) - diff(u,y))/2  # Rotación en un punto.

In [19]:
e1 =   u.subs([(x, 0)]) - 0  # =0
e2 =   v.subs([(x, 0)]) - 0  # =0
e3 = rot.subs([(x, 0)]) - 0  # =0

In [20]:
u0v0w0 = solve([e1,e2,e3],[u0, v0, w0])

Reemplazamos las constantes de integración:

In [21]:
u = u.subs(u0v0w0)
v = v.subs(u0v0w0)

Obtenemos los desplazamientos de la viga:

In [22]:
imprimir(r"u(x,y) = ", factor(u))

<IPython.core.display.Math object>

In [23]:
imprimir(r"v(x,y) = ", factor(v))

<IPython.core.display.Math object>

Por último, la deflexión vertical de la viga en el punto $x=L$, $y=L$ es:

In [24]:
v_L_0 = v.subs([(x,L), (y,0)])
imprimir(r"v(0,0) = ", factor(v_L_0))

<IPython.core.display.Math object>

Reemplazamos $\frac{2tc^3}{3}$ por $I$:

In [25]:
I  = symbols("I")
Ib = 2*t*c**3/3
v_L_0 = v_L_0*Ib/I  # Es equivalente a multiplicar por uno.

Primera forma de factorizar la expresión:

In [26]:
ec1 = Add(expand(v_L_0/(Rational(1,8)*q*L**4/(E*I))) * (Rational(1,8)*q*L**4/(E*I)),
          evaluate=False)
imprimir(r"v(0,0) =", ec1)

<IPython.core.display.Math object>

Segunda forma de factorizar la expresión:

In [27]:
_2c = symbols("(2c)")
ec2 = collect(ec1.subs(c, _2c/2), _2c**2/L**2)
imprimir(r"v(0,0) =", ec2)

<IPython.core.display.Math object>

En este caso, el parámetro $h/L$ viene representado de forma equivalente por $2c/L$. Por lo tanto, el % de variación respecto a la teoría de Euler-Bernoulli (revisar [este programa](https://github.com/jnramirezg/medio_continuo/blob/main/codigo/29-modelo_viga_voladizo.ipynb)) viene dado por:
$$\left(\frac{3\nu}{2}+\frac{6}{5}\right)\left(\frac{2c}{L}\right)^2 = \left(\frac{3\nu}{2}+\frac{6}{5}\right)\left(\frac{h}{L}\right)^2$$

Suponiendo $\nu=0.3$, por ejemplo, para $2c/L=h/L=[0.05, 0.1, 0.25]$:

In [28]:
nu=0.3

In [29]:
h_L=0.05
(3*nu/2+6/5)*(h_L)**2

In [30]:
h_L=0.1
(3*nu/2+6/5)*(h_L)**2

In [31]:
h_L=0.25
(3*nu/2+6/5)*(h_L)**2